In [31]:
# For each question, predict the correctness of every student response as always either correct or incorrect, based on 
# whichever outcome is more common for that question in the training data.

import pandas as pd
import numpy as np
import argparse
import os
from scipy.sparse import csc_matrix

In [32]:
base_dir = '../Data/'

input_dir = base_dir + "data/train_data"
train_data = base_dir + "data/train_data/train_task_1_2.csv"
test_data_1 = base_dir + "data/test_data/test_public_answers_task_1.csv"
# submission_template_dir = base_dir + "data/train_data"
# submission_template_file = base_dir + "data/train_data"
# output_dir = base_dir + "data/train_data"
# output_file = base_dir + "data/train_data"



In [33]:
import pandas as pd
df = pd.read_csv(train_data)

In [34]:
df.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,16997,65967,12453206,0,4,2
1,16531,62121,15686710,1,1,1
2,15911,50013,13598796,0,3,1
3,1701,104909,10511925,0,4,3
4,22896,21748,941747,0,1,4


In [35]:
# df.groupby('UserId').sum()
df['UserId'].value_counts().describe()

count    118971.000000
mean        133.375781
std         127.762689
min          29.000000
25%          57.000000
50%          88.000000
75%         159.000000
max        2445.000000
Name: UserId, dtype: float64

In [36]:
df['QuestionId'].value_counts().describe()

count    27613.000000
mean       574.651432
std        654.449058
min         34.000000
25%        109.000000
50%        281.000000
75%        850.000000
max       6868.000000
Name: QuestionId, dtype: float64

In [37]:
df_test = pd.read_csv(test_data_1)

In [38]:
df_test.head()

,QuestionId,UserId,AnswerId,IsCorrect
0,10856,5518,1127409,1
1,14766,41565,19673056,1
2,2829,97267,19426951,0
3,26464,100879,4929721,1
4,24436,87722,2927607,0


In [39]:
tr_uid = set(df['UserId'].values)
test_uid = set(df_test['UserId'].values)

In [40]:
tr_qid = set(df['QuestionId'].values)
test_qid = set(df_test['QuestionId'].values)

In [41]:
# len(test_uid - tr_uid)
len(test_qid - tr_qid)

0

# Checks

In [42]:
ques_ids = set(df['QuestionId'].values)
user_ids = set(df['UserId'].values)

In [43]:
print(min(ques_ids))
print(max(ques_ids))
print(len(ques_ids))
print(min(user_ids))
print(max(user_ids))
print(len(user_ids))

0
27612
27613
0
118970
118971


In [44]:
#TODO: Check if same QuestionID, UserId pair appears more than once

# Method 2

In [45]:
mtr = df[['QuestionId','UserId','IsCorrect']]

In [46]:
rows = df['UserId'].values
cols = df['QuestionId'].values
data = df['IsCorrect'].values
data = data*0.1-0.1

In [47]:
ind = 4
print(rows[ind],cols[ind],data[ind])
print(mtr.values[ind])

21748 22896 -0.1
[22896 21748     0]


In [48]:
sparse_mtr = csc_matrix((data, (rows, cols)), dtype=np.int8)

In [49]:
from sparsesvd import sparsesvd
K = 150
print(type(sparse_mtr))
U, s, Vt = sparsesvd(sparse_mtr, K)

<class 'scipy.sparse.csc.csc_matrix'>


ValueError: Invalid shape in axis 0: 0.

In [ ]:
S = np.diag(s)

In [ ]:
right_term = np.matmul(S,Vt)
right_term.shape
# U.shape

In [ ]:
def get_pred(qid,uid):
    user_features = U[:,uid]
    question_features = right_term[:,qid]
    score = np.dot(user_features,question_features)
    return np.sign(score)

In [ ]:
df_pred = df_test.copy()
print(df_pred.head())
df_pred['Prediction'] = df_pred.apply(lambda row: get_pred(row['QuestionId'],row['UserId']), axis=1)

In [ ]:
df_pred['Prediction'] = (df_pred['Prediction'] + 1)/2

In [ ]:
score_num = (df_pred['Prediction'] == df_pred['IsCorrect']).sum()
score_den = len(df_pred)
score = score_num / score_den
score